In [63]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from sklearn.linear_model import LinearRegression

In [64]:
data = pd.read_csv('train.csv') # Loading training data 

In [65]:
len(data.columns)

17

## Reducing Memory Usage

In [66]:
def reduce_memory_usage(df):
    print("Memory Usage Before Optimization:")
    print(df.memory_usage(deep=True).sum() / (1024 ** 2), "MB")

    # Iterate through each column in the DataFrame
    for col in df.columns:
        col_type = df[col].dtype

        # Check if the column's data type is numeric
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Check if the column's data type is an integer
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
                    
    # Display the memory usage after optimization
    print("\nMemory Usage After Optimization:")
    print(df.memory_usage(deep=True).sum() / (1024 ** 2), "MB")
    return df

data = reduce_memory_usage(data)

Memory Usage Before Optimization:
974.2033262252808 MB

Memory Usage After Optimization:
599.5538015365601 MB


In [67]:
data.head() # Reading first few examples 

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3.180603e+06,1,0.999812,13380277.00,NaN,NaN,0.999812,60651.500000,1.000026,8493.030273,1.0,-3.029704,0,0_0_0
1,1,0,0,1.666039e+05,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.040039,1.000660,20605.089844,1.0,-5.519986,0,0_0_1
2,2,0,0,3.028799e+05,-1,0.999561,1819368.00,NaN,NaN,0.999403,37956.000000,1.000298,18995.000000,1.0,-8.389950,0,0_0_2
3,3,0,0,1.191768e+07,-1,1.000171,18389746.00,NaN,NaN,0.999999,2324.899902,1.000214,479032.406250,1.0,-4.010201,0,0_0_3
4,4,0,0,4.475500e+05,-1,0.999532,17860614.00,NaN,NaN,0.999394,16485.539062,1.000016,434.100006,1.0,-7.349849,0,0_0_4


## Rolling Averages for Missing Elements

In [68]:
print(data.isnull().sum()) # Displaying columns with missing data

stock_id                         0
date_id                          0
seconds_in_bucket                0
imbalance_size                 220
imbalance_buy_sell_flag          0
reference_price                220
matched_size                   220
far_price                  2894342
near_price                 2857180
bid_price                      220
bid_size                         0
ask_price                      220
ask_size                         0
wap                            220
target                          88
time_id                          0
row_id                           0
dtype: int64


In [69]:
# Creating list of columns with missing data 
columns_with_missing_values = ['imbalance_size', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'ask_price', 'wap', 'target']

In [70]:
# Checking if any columns start with missing values

first_missing_index_imbalance_size = data['imbalance_size'].isnull().idxmax() if data['imbalance_size'].isnull().any() else None
first_missing_index_reference_price = data['reference_price'].isnull().idxmax() if data['reference_price'].isnull().any() else None
first_missing_index_matched_size = data['matched_size'].isnull().idxmax() if data['matched_size'].isnull().any() else None
first_missing_index_far_price = data['far_price'].isnull().idxmax() if data['far_price'].isnull().any() else None
first_missing_index_near_price = data['near_price'].isnull().idxmax() if data['near_price'].isnull().any() else None
first_missing_index_bid_price = data['bid_price'].isnull().idxmax() if data['bid_price'].isnull().any() else None
first_missing_index_ask_price = data['ask_price'].isnull().idxmax() if data['ask_price'].isnull().any() else None
first_missing_index_wap = data['wap'].isnull().idxmax() if data['wap'].isnull().any() else None
first_missing_index_target = data['target'].isnull().idxmax() if data['target'].isnull().any() else None


first_missing_index_imbalance_size, first_missing_index_reference_price, first_missing_index_matched_size, first_missing_index_far_price, first_missing_index_near_price, first_missing_index_bid_price, first_missing_index_ask_price, first_missing_index_wap, first_missing_index_target

(369508, 369508, 369508, 0, 0, 369508, 369508, 369508, 369508)

## 'far_price' and 'near_price' columns start with missing values so we need to handle these edge cases using -1 imputations

In [71]:
for col in columns_with_missing_values:
    
    data[col] = data[col].fillna(-1)

In [72]:
print(data.isnull().sum()) # Checking that all missing elements are no longer empty

stock_id                   0
date_id                    0
seconds_in_bucket          0
imbalance_size             0
imbalance_buy_sell_flag    0
reference_price            0
matched_size               0
far_price                  0
near_price                 0
bid_price                  0
bid_size                   0
ask_price                  0
ask_size                   0
wap                        0
target                     0
time_id                    0
row_id                     0
dtype: int64


## Creating New Features

In [73]:
data['bid_ref_price_diff'] = data['bid_price'] - data['reference_price']
data['bid_ref_price_ratio'] = data['bid_price'] / data['reference_price']
data['ref_price_ma_5'] = data['reference_price'].rolling(window=5).mean()
data['price_momentum'] = data['reference_price'].diff()
data['volume_weighted_price'] = (data['reference_price'] * data['matched_size']) / data['matched_size'].sum()
data['bid_size_volume_ratio'] = data['bid_size'] / data['matched_size']
data['imbalance_volume_interaction'] = data['imbalance_size'] * data['matched_size']
data['day_of_week'] = data['date_id'] % 7
data['hour_of_day'] = (data['seconds_in_bucket'] // 3600) % 24
data['price_volatility'] = data['reference_price'].rolling(window=5).std()

In [74]:
print(data.isnull().sum())

stock_id                        0
date_id                         0
seconds_in_bucket               0
imbalance_size                  0
imbalance_buy_sell_flag         0
reference_price                 0
matched_size                    0
far_price                       0
near_price                      0
bid_price                       0
bid_size                        0
ask_price                       0
ask_size                        0
wap                             0
target                          0
time_id                         0
row_id                          0
bid_ref_price_diff              0
bid_ref_price_ratio             0
ref_price_ma_5                  4
price_momentum                  1
volume_weighted_price           0
bid_size_volume_ratio           0
imbalance_volume_interaction    0
day_of_week                     0
hour_of_day                     0
price_volatility                4
dtype: int64


In [75]:
new_columns_with_missing_values = ['ref_price_ma_5', 'price_momentum', 'price_volatility']

In [76]:
window_size = 5 # Number of elements before current missing element to take an average of 

for col in new_columns_with_missing_values:
    data[col] = data[col].fillna(-1) # Using backward fill 

In [77]:
print(data.isnull().sum())

stock_id                        0
date_id                         0
seconds_in_bucket               0
imbalance_size                  0
imbalance_buy_sell_flag         0
reference_price                 0
matched_size                    0
far_price                       0
near_price                      0
bid_price                       0
bid_size                        0
ask_price                       0
ask_size                        0
wap                             0
target                          0
time_id                         0
row_id                          0
bid_ref_price_diff              0
bid_ref_price_ratio             0
ref_price_ma_5                  0
price_momentum                  0
volume_weighted_price           0
bid_size_volume_ratio           0
imbalance_volume_interaction    0
day_of_week                     0
hour_of_day                     0
price_volatility                0
dtype: int64
